In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense ,Flatten ,BatchNormalization ,GlobalMaxPool2D
from tensorflow.keras.optimizers import Adam 


In [3]:
train_df = pd.read_csv('styles.csv', usecols=['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName'])

In [4]:
train_df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [5]:
train_df["id"]=train_df["id"].astype(str)
train_df["id"] = train_df["id"].apply(lambda x: x + ".jpg")

In [6]:
train_df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970.jpg,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt
1,39386.jpg,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans
2,59263.jpg,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch
3,21379.jpg,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants
4,53759.jpg,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt


In [7]:
dataset = train_df.groupby('articleType').sample(frac =0.2)

In [8]:
dataset.shape

(8885, 10)

In [64]:
datagen = ImageDataGenerator(
            # rescale=1./255.,
            # width_shift_range=0.2,
            # height_shift_range=0.2,
            # horizontal_flip=True,
            dtype ='float32',
            preprocessing_function=preprocess_input,
            validation_split = 0.30
            )

In [88]:
train_generator = datagen.flow_from_dataframe(
            dataframe=train_df,
            directory="images",
            x_col="id",
            y_col="articleType",
            subset="training",
            batch_size=32,
            shuffle=True,
            color_mode='rgb',
            class_mode="categorical",
            target_size=(224,224))
print('Train generator created')

Found 31109 validated image filenames belonging to 142 classes.
Train generator created


/Users/samuelpradhan/anaconda3/lib/python3.11/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 5 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  warnings.warn(


In [89]:
val_generator = datagen.flow_from_dataframe(
            dataframe=train_df,
            directory="images",
            x_col="id",
            y_col="articleType",
            subset="validation",
            batch_size=32,
            shuffle=True,
            class_mode="categorical",
            target_size=(224,224)
)

Found 13332 validated image filenames belonging to 142 classes.


/Users/samuelpradhan/anaconda3/lib/python3.11/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 5 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  warnings.warn(


In [90]:
conv_base = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
conv_base.trainable = False

In [91]:
model=Sequential([
                  conv_base,
                  GlobalMaxPool2D(),
                  Flatten(),
                  Dense(256,activation='relu'),
                  Dense(142,activation='softmax')
])

In [96]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling2d_9          │ (None, 2048)           │             0 │
│ (GlobalMaxPooling2D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 142)            │        36,494 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,270,828 (96.40 MB)

 Trainable params: 561,038 (2.14 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

 Optimizer params: 1,122,078 (4.28 MB)

In [94]:
model.compile(Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [95]:
history=model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5


/Users/samuelpradhan/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


973/973 ━━━━━━━━━━━━━━━━━━━━ 392s 392ms/step - accuracy: 0.5946 - loss: 2.4648 - val_accuracy: 0.7224 - val_loss: 0.9694
Epoch 2/5
973/973 ━━━━━━━━━━━━━━━━━━━━ 375s 384ms/step - accuracy: 0.7783 - loss: 0.7564 - val_accuracy: 0.7793 - val_loss: 0.7852
Epoch 3/5
 75/973 ━━━━━━━━━━━━━━━━━━━━ 4:42 315ms/step - accuracy: 0.8216 - loss: 0.5764

KeyboardInterrupt: 

In [25]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()

417/417 ━━━━━━━━━━━━━━━━━━━━ 56s 135ms/step - accuracy: 0.2052 - loss: 4.8613
Training Loss: 4.834134101867676
Training Accuracy: 0.20651358366012573


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()